# Installing necessary MATLAB modules for Python

In [6]:
!pip install matlab --user
!pip install matlabengine --user

# Run above cell twice to see the correct path for your machine!

Then you can set the correct path in below cell.

In [3]:
!setx PATH "%PATH%;c:\users\exx520\appdata\roaming\python\python37\site-packages"


SUCCESS: Specified value was saved.


# Before continuing, please restart the kernel from the upper menu "Kernel>Restart"

In [7]:
import matlab.engine

In [8]:
eng = matlab.engine.start_matlab()

In [9]:
tf = eng.isprime(37)
print(tf)

True


In [16]:
import pandas as pd
import numpy as np

In [21]:
df = pd.read_csv("group1_hkg_0.txt")

df=df.drop('isSnow', axis=1)
df=df.drop('isFog', axis=1)
df=df.drop('isHail', axis=1)
df=df.drop('budget', axis=1)

train_percentage = 0.8 # you can set the training percentage here
train, test = np.split(df.sample(frac=1, random_state=42), [int(train_percentage*len(df))])

In [22]:
print(len(train))
print(len(test))
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

4412
1103


In [37]:
train.tail()

,depArr,distance,angle_error,distance_long,operation_mode,angle_sum,QDepDep,QDepArr,QArrDep,QArrArr,...,isMist,isHaze,aircraft_weight,AvgSpdLast5Dep,AvgSpdLast5Arr,AvgSpdLast5,AvgSpdLast10Dep,AvgSpdLast10Arr,AvgSpdLast10,TaxiTime
3280,1,2960.340074,0,2210.343834,0,689.93,0,0,2,0,...,0,0,3,325.653740,389.695682,308.222966,321.612499,403.986356,337.685054,6.416667
3186,1,3271.852438,0,1754.248652,0,690.21,0,0,3,1,...,0,0,2,289.046775,421.540112,292.513170,288.621663,527.507882,278.569839,7.633333
3953,0,4716.078401,2,3686.083841,0,420.36,3,3,0,0,...,0,0,3,302.917748,505.960072,381.631830,288.513322,532.039615,365.777681,17.133333
2768,0,2817.026957,0,2073.171319,0,280.57,2,0,0,0,...,0,0,2,296.611149,485.334790,354.095154,319.145784,506.045132,373.153209,14.466667
4223,0,1761.669890,0,906.824376,3,1001.81,4,5,0,0,...,0,0,3,137.310275,485.669181,292.720848,185.730863,506.346005,259.013143,13.100000


% TRAINING & MODEL GENERATION SCRIPT - SCRIPT #1

% reading training data file
data=readtable('train.csv');
dataArr=table2array(data);
datain=dataArr(:,1:size(dataArr,2)-1);
dataout=dataArr(:,size(dataArr,2));

% running subtractive clustering and generate the fis model
fisOpt = genfisOptions("SubtractiveClustering",...
    "ClusterInfluenceRange",0.5);
fis = genfis(datain,dataout,fisOpt);

% print out the fis model
writeFIS(fis,'fisFile')

% evaluate the fis model
fuzout = evalfis(fis,datain);

% printing out the predictions
T = array2table(fuzout);
writetable(T, 'fuzOut.csv');

% calculate RMSE
trnRMSE = norm(fuzout-dataout)/sqrt(length(fuzout));

% print out the RMSE
fileID=fopen('trainRMSE-result.txt','wt');
fprintf(fileID, '%e\n', trnRMSE);
fclose(fileID);

In [19]:
# this runs trainingScript.m MATLAB script for training and generating the prediction model
eng.trainingScript(nargout=0);

In [20]:
# this runs testingScript.m MATLAB script for testing the prediction model
eng.testingScript(nargout=0);

In [24]:
# this prints out the root-mean-square error for the training
file1 = open("trainRMSE-result.txt", "r")
print("Training RMSE: ", file1.readline())
file1.close()

Training RMSE:  1.607678e+00



In [23]:
# this prints out the root-mean-square error for the testing
file2 = open("testRMSE-result.txt", "r")
print("Testing RMSE: ", file2.readline())
file2.close()

Testing RMSE:  1.748966e+00

